# Tutorial & Widget Experiments

> This notebook is primarily being used to experiment with how to use the `pymbe` widgets.
>
> It should be fairly "clean" until the `Scratch Pad` section.
>

In the future, this notebook will be cleaned up and renamed "Tutorial".

## 1. Import `pymbe` and create a new user interface

In [ ]:
import pymbe.api as pm

In [ ]:
pm.UI.new(host_url="http://sysml2-sst.intercax.com")

## 2. Use the widget
...or automatically load the `Kerbal` model using the cell below

In [ ]:
ui, *_ = _.children
ui.client.project_selector.value = ui.client.project_selector.options["Kerbal"]
ui.client._download_elements()

...and, for example, select some edge types to filter down the diagram

In [ ]:
EDGE_TYPES_TO_SELECT =  ("Superclassing", "FeatureTyping", "FeatureMembership")
lpg.edge_type_selector.value = [
    edges
    for key, edges in lpg.edge_type_selector.options.items()
    if any(
        key.startswith(edge_type)
        for edge_type in EDGE_TYPES_TO_SELECT
    )
]
lpg._update_diagram_graph()

# Interpretation

> This is where we are refactoring the $M_0$ interpretation...

In [ ]:
def pop_tree(*_):
    print("before")
    DockPop(
        [ipyw.VBox([project.tree])],
        layout=dict(height="60vh"),
    )
    print("after")

tree = ipyw.Button(
    icon="poll-h",
    on_click=pop_tree,
    tooltip="Project Tree",
    layout=dict(width="40px"),
)
tree.on_click(pop_tree)
buttons = ipyw.VBox([tree])

DockPop([ipyw.VBox([client.widget, buttons])], mode="split-right")

In [ ]:
class SafeDict(dict):

    __DEFAULT__ = dict()

    def __init__(self, iterable, default=None):
        self.__default__ = default

    def get(self, item, default=dict()):
        return super().get(item, default) or self.__DEFAULT__


a = SafeDict()
a["as"] = "if"
a.get("b")

In [ ]:
from pymbe.interpretation import make_banded_featuring_graph


def retrieve_element(elements: dict, element: (str, dict), strict: bool = True) -> dict:
    input_element = element
    if isinstance(element, str):
        element = elements.get(element, None)
    elif not isinstance(element, dict):
        raise ValueError(f"Failed to process element: '{input_element}'")
    if strict and element is None:
        raise ValueError(f"Failed to process element: '{input_element}'")
    return element


def get_metaclass(elements, element) -> str:
    element = retrieve_element(elements, element)
    return element.get("@type", None)


def get_feature_upper_multiplicity(elements: dict, feature: (dict, str)) -> int:
    feature = retrieve_element(elements, feature)

    multiplicity = elements.get((feature.get("multiplicity", {}) or {}).get("@id", None), None)
    if multiplicity is None:
        return None
    upper_bound = elements.get((multiplicity.get("upperBound", {}) or {}).get("@id", {}), None)
    if upper_bound is None:
        return None
    return upper_bound.get("value", None)


def roll_up_upper_multiplicities(lpg):
    banded_featuring_graph = make_banded_featuring_graph(lpg)

    banded_roots = [
        banded_featuring_graph.nodes[node]
        for node in banded_featuring_graph
        if banded_featuring_graph.out_degree(node) == 0
    ]
    
    for part_usage in 

elements = lpg.elements_by_id

features = [
    id_
    for id_, data in elements.items()
    if data["@type"] == "Feature"
]

for feature in features:
    upper_multiplicity = get_feature_upper_multiplicity(elements, feature)
    if upper_multiplicity:
        print(elements[feature]["qualifiedName"], upper_multiplicity)

# Troubleshooting

## Fix linking issue with diagram element selector

In [ ]:
tree.selected_nodes

# Subgraphing to Node Reach

# Scratch Pad
> **WARNING**: Anything below this point is just for experimentation purposes

In [ ]:
# Create a widget to see the qualified name of the elements nodes selected
import ipywidgets as ipyw

out = ipyw.Output(layout={'border': '1px solid black'})

def update_node_selections(*_):
    out.outputs = []
    with out:
        [
            print(element["qualifiedName"])
            for element in lpg.selected_nodes_by_type
        ]
            
lpg.node_type_selector.observe(update_node_selections, "value")

out

## Project Tree / LPG element selection Experiments

In [ ]:
from copy import deepcopy

#data = deepcopy(project.element_data.outputs[0])
# data
project.element_data.clear_output()
data["data"]["application/json"] = {"a": 10, "2": "c"}
project.element_data.outputs = [data]

import json

project.element_data.outputs[0]["data"]["application/json"] = {"a": 10, "b": 100}
project.element_data.outputs = [{**project.element_data.outputs[0]}]
project.element_data.clear_output()

lpg.diagram.elk_app.diagram.selected = ('275e842f-85d4-4e30-accc-3f8d826bfd63', 'd150a7ef-15a2-428d-a1f6-bdc5b2074426', '4ca57b96-7b73-47c7-a69e-d204d92d8773',)

project.tree.nodes[0].selected = True
project.tree.
lpg.diagram.elk_app.toolbar.layout.flex = "0"

def safe_linker(elements=None):
    elements = elements or []
    return [
        element._identifier
        for element in elements
    ]

trt.dlink(
    (project.tree, "selected_nodes"),
    (lpg.diagram.elk_app.diagram, "selected"),
    safe_linker,
)

lpg.diagram.elk_app.toolbar._model_name = "BoxModel"
lpg.diagram.elk_app.toolbar._model_name = "BoxView"

## RDF Experiments

In [ ]:
def process_edge(source, target, edge_type, data=None):
    if edge_type in ("FeatureTyping", "FeatureMembership"):
        source, target = target, source
    return [source, target, data]

graph = nx.DiGraph()
graph.add_edges_from([
    process_edge(source, target, edge_type, data)
    for (source, target, edge_type), data in dict(client.lpg.graph.edges).items()
    if edge_type in ("Superclassing", "FeatureTyping", "FeatureMembership")
])
graph

In [ ]:
diagram.graph = graph

In [ ]:
diagram = client.lpg.make_diagram(
    graph=client.lpg.subgraph(edge_types=("Superclassing", "FeatureTyping^-1", "FeatureMembership^-1"))
)
elk_app, *_ = diagram.children
diagram

In [ ]:
# Select the root node in the diagram...
# ... or you can manually select one yourself
elk_app.selected = "5260380b-6fda-43cc-993f-5df58868edbb",

In [ ]:
first_element_selected, *_ = elk_app.selected
client.elements_by_id[first_element_selected]

# Parse JSON-LD into RDF

In [ ]:
import rdflib
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
import networkx as nx
import matplotlib.pyplot as plt

result = client.rdf.graph
# result = g.parse(url, format='turtle')

G = rdflib_to_networkx_multidigraph(result)

# Plot Networkx instance of RDF Graph
pos = nx.spring_layout(G, scale=2)
edge_labels = nx.get_edge_attributes(G, 'r')

In [ ]:
ax = plt.figure(figsize=(50,30)).gca();
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, ax=ax)
nx.draw(G, with_labels=True, ax=ax)

# TODOs
1. Finish fleshing out the process in the `Kerbal Model.ipynb`
2. Modify the subgraph generator so it can take the value from the `Type Selector` directly
3. Improve the ipyelk diagram widget (may need to make improvements to `ipyelk`)
   * Add arrows
   * Add compartments
   * Fix layout
   * Add widget to see node details
4. Finalize the RDF formulation